# *Cartesius* energy system model - version 1



In [1]:
# Importing libraries: ------
import pyomo.environ as pyo
import numpy as np
import pandas as pp

# Import data from excel: ---
def readInputFile(filename):
    EV_data = pp.read_excel(filename, sheet_name= 'EV_data', index_col=0)
    time = pp.read_excel(filename, sheet_name= 'EV_time', index_col=0)
    EV_chargers = pp.read_excel(filename, sheet_name= 'EV_chargers', index_col=0)
    grid_connection = pp.read_excel(filename, sheet_name= 'grid_connection', index_col=0)
    transformers = pp.read_excel(filename, sheet_name= 'transformers', index_col=0)
    HP = pp.read_excel(filename, sheet_name= 'HP', index_col=0)
    load = pp.read_excel(filename, sheet_name= 'load', index_col=0)
    BESS = pp.read_excel(filename, sheet_name= 'BESS', index_col=0)
    return {'EV_data':EV_data, 'time':time, 'EV_chargers':EV_chargers, 'grid_connection':grid_connection, 'transformers':transformers, 'HP':HP, 'load':load, 'BESS':BESS}

filename = 'Initialized_Data'
data = readInputFile(filename)
# ----------------------------
EV_data = data['EV_data']
time = data['time']
EV_chargers = data['EV_chargers']
grid_connection = data['grid_connection']
transformers = data['transformers']
HP = data['HP']
load = data['load']
BESS = data['BESS']
# ----------------------------

# Create a model: ------------
model = pyo.ConcreteModel()


# Sets: ----------------------
model.T = pyo.Set(initialize=time.index, ordered=True)                   # Time [h1, ..., h24]
model.S = pyo.Set(initialize=EV_data.index, ordered=True)                   # EVs [EV1, ..., EVs]
model.CH = pyo.Set(initialize=EV_chargers.index, ordered=True)              # Chargers [CH1, ..., CHch]
model.L = pyo.Set(initialize=load.index, ordered=True)                      # Load [L1, ..., Ll]
model.B = pyo.Set(initialize=BESS.index, ordered=True)                      # BESS [B1, ..., Bb]
model.H = pyo.Set(initialize=HP.index, ordered=True)                        # Heat Pumps [HP1, ..., HPh]
model.G = pyo.Set(initialize=grid_connection.index, ordered=True)           # Grid Connection [POC1, ..., POCg]
model.TF = pyo.Set(initialize=transformers.index, ordered=True)             # Transformers [TRAFO1, ..., TRAFOtf]


# ----------------------------
# Parameters:
# ----------------------------

# EVs:
model.EV.P = pyo.Param(model.S, model.T)                                    # Max power the s-th EV can deliver at time t [kW]
model.EV.E = pyo.Param(model.S)                                             # Energy capacity of the s-th EV [kWh]
model.EV.Ta = pyo.Param(model.S)                                            # Arrival time of the s-th EV [h]*
model.EV.Td = pyo.Param(model.S)                                            # Departure time of the s-th EV [h]*





